# **Youtube Informative-video Summerizer**

This python app let's you summerize youtube videos that contains information-sharing-through-talking, like someone talking about a subject, someone sharing a life advice, a podcast etc.

We extract the transcipt analyize it with an LLM to summerize and create summerization and analysis.


> We use youtube_transcript_api which allows you to get the transcript text of any youtube video.

> Results however are not ideal for our use case since it does not provide who says what in case of more than one speaker. it only provide one giant string of all the words said in the video respectivly with some noise.










In [ ]:
#!pip install youtube-transcript-api

In [35]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter, SRTFormatter
import re
from openai import OpenAI
from google.colab import userdata # dotenv equevilant for google colab
from IPython.display import Markdown, display, update_display

In [36]:
ytt = YouTubeTranscriptApi()
formatter = TextFormatter() # --> Plain text
# formatter = SRTFormatter() # --> With timestamps

openai_api_key = userdata.get('OPENAI_TOKEN')
openai_client = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-mini"

In [37]:
def get_video_id(url):
    """Extracts video ID from a YouTube URL."""
    regex = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    match = re.search(regex, url)
    if match:
        return match.group(1)
    raise ValueError("Invalid YouTube URL")


def get_transcript(url):
    video_id = get_video_id(url)
    fetched_transcript = ytt.fetch(video_id)
    # ^ defaults to English transcript, for other language use:
    # fetched = ytt.fetch(video_id, languages=['de', 'en'])
    transcript_text = formatter.format_transcript(fetched_transcript)
    return transcript_text

In [38]:
system_prompt = """You are an expert assistant specialized in analyzing podcast transcripts. You will be given the full transcript of a YouTube podcast episode.

Your task is to extract and summarize the main views or arguments presented in the podcast. For each view or argument, also identify and list any supporting evidence such as:

- Facts or statistics
- Academic studies or research
- Theories or philosophical frameworks
- Anecdotes or personal experiences
- Expert opinions or quotes

Recognize off topic segments and adds and igrone them.

Structure your output in a clear and concise format.

Output Format:

Podcast Summary:

1. View/Argument:
   - Description: [Summarize the view or claim in 1-2 sentences.]
   - Supporting Evidence:
     • [Fact, study, or reasoning #1]
     • [Fact, study, or reasoning #2]
     • [Optional counterarguments or nuances, if any]

2. View/Argument:
   - Description: [...]
   - Supporting Evidence:
     • [...]

Guidelines:
- Only include major views or arguments that are discussed in depth.
- Paraphrase in clear, neutral, and objective language.
- Do not include filler, small talk, or off-topic segments.
- If a claim lacks explicit evidence, note it as “No clear supporting evidence provided.”

Always respond and orginize your response using Markdow.
"""


In [39]:
def get_user_prompt(title,url):
    prompt = f"Following is a transcript for a podcast titled '{title}' \n"
    prompt += "Carefully read through this content, analyse and summerize it as told, respond in Markdown."
    prompt += "\nTranscript: \n\n"
    prompt += get_transcript(url)
    return prompt

In [40]:
# user_prompt = get_user_prompt()
def summerize_video(title,url):
    user_prompt = get_user_prompt(title,url)
    stream = openai_client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream = True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
summerize_video("Anti-Aging Expert: Missing This Vitamin Is As Bad As Smoking! The Truth About Creatine!","https://www.youtube.com/watch?v=JCTb3QSrGMQ")

# Podcast Summary: "Anti-Aging Expert: Missing This Vitamin Is As Bad As Smoking! The Truth About Creatine!"

1. **View/Argument: Vitamin D and Health Risks**
   - **Description:** Vitamin D deficiency significantly increases the risk of dementia and various health issues, yet many individuals are unaware of its critical importance.
   - **Supporting Evidence:**
     - Vitamin D deficiency can raise dementia risk by 80%.
     - Individuals with adequate vitamin D have a 40% reduced risk of dementia and experience better cognitive function.

2. **View/Argument: Role of Lifestyle in Aging**
   - **Description:** Lifestyle choices account for over 70% of aging effects, with exercise and nutrition being key factors in improving longevity and health.
   - **Supporting Evidence:**
     - Studies show participants involved in regular exercise did not experience hippocampal shrinkage, but rather an increase in size.
     - Exercise is equated to a miracle drug for its extensive health benefits, as highlighted by unquantifiable positive effects when compared to medications.

3. **View/Argument: Importance of Magnesium**
   - **Description:** Magnesium is crucial for cellular function, metabolism, and reducing cancer risk, yet nearly half the U.S. population is magnesium deficient.
   - **Supporting Evidence:**
     - Individuals with the highest magnesium levels have a 40% lower all-cause mortality.
     - A 24% increase in pancreatic cancer incidents is associated with every 100 mg decrease in magnesium intake.

4. **View/Argument: Benefits of Creatine in Brain Health**
   - **Description:** Creatine isn't just beneficial for muscle health but also shows promise for enhancing cognitive performance, especially under stress or sleep deprivation.
   - **Supporting Evidence:**
     - A study found that creatine can negate cognitive deficits caused by 21 hours of sleep deprivation.
     - Users often report improved focus and energy levels when supplementing with creatine regularly.

5. **View/Argument: Exercise and Hormonal Benefits**
   - **Description:** Regular exercise, especially high-intensity interval training, can reverse heart aging and improve mental health markers.
   - **Supporting Evidence:**
     - Participants in an intensive exercise program showed heart structures that were more akin to those of individuals two decades younger.
     - High-intensity workouts were shown to improve cognition and neuroplasticity due to the metabolic changes they induce.

6. **View/Argument: Impact of Nutrition on Cognitive Function**
   - **Description:** A healthy diet rich in omega-3 fatty acids, vitamins D and other nutrients is essential for maintaining cognitive function and overall health.
   - **Supporting Evidence:**
     - Adequate omega-3 intake has been linked to a 5-year increase in life expectancy.
     - Regular consumption of nutrient-rich foods, such as blueberries and dark leafy greens, supports cognition and potentially reduces the risk of neurodegenerative diseases.

7. **View/Argument: The Importance of Autophagy**
   - **Description:** Fasting promotes autophagy, a cellular cleaning process that can protect against diseases and improve health.
   - **Supporting Evidence:**
     - Studies suggest that fasting for 16 hours can activate autophagy and contribute to cellular repair processes.

8. **View/Argument: Intermittent Fasting and Health Improvements**
   - **Description:** Intermittent fasting can improve metabolic parameters and cognitive performance while providing health benefits beyond simple calorie restriction.
   - **Supporting Evidence:**
     - Individuals practicing intermittent fasting showed improved glucose regulation compared to those restricting calories alone without fasting.

9. **View/Argument: Microplastics and Health Risks**
   - **Description:** The pervasive presence of microplastics in everyday products poses health risks that are not widely recognized.
   - **Supporting Evidence:**
     - Common items, such as paper coffee cups and plastic water bottles, can release harmful chemicals, leading to increased levels of substances like BPA in beverages.

By summarizing these key points, the podcast emphasizes the interconnectedness of nutrition, exercise, and mental well-being in managing aging and chronic diseases. Additionally, it highlights emerging research on creatine, fasting, and environmental health risks that affect longevity and quality of life.